In [1]:
!pip install gradio tensorflow transformers soundfile gdown --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.6 MB/s eta 0:00:00


In [2]:
import gradio as gr
import torch
import numpy as np
import soundfile as sf
from tensorflow.keras.models import load_model
from transformers import ASTFeatureExtractor


In [3]:
# Scarica il modello da Google Drive
!gdown 1SZcsg0S16GJByrbWvOlIAlpYhmV5sYXG -O ast_fcnn_fold_1_embeddings.keras


Downloading...
From: https://drive.google.com/uc?id=1SZcsg0S16GJByrbWvOlIAlpYhmV5sYXG
To: /content/ast_fcnn_fold_1_embeddings.keras
100% 6.92M/6.92M [00:00<00:00, 17.5MB/s]


In [4]:
# Scaricare il CSV in formato raw dal repository GitHub
!wget https://raw.githubusercontent.com/karolpiczak/ESC-50/master/meta/esc50.csv -O esc50.csv
import pandas as pd

metadata = pd.read_csv("esc50.csv")



--2025-01-18 14:27:02--  https://raw.githubusercontent.com/karolpiczak/ESC-50/master/meta/esc50.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93742 (92K) [text/plain]
Saving to: ‘esc50.csv’

esc50.csv           100%[===================>]  91.54K  --.-KB/s    in 0.004s  

2025-01-18 14:27:03 (23.6 MB/s) - ‘esc50.csv’ saved [93742/93742]



In [5]:

# Load the downloaded model
model = load_model("ast_fcnn_fold_1_embeddings.keras")


print("Model loaded successfully.")


Model loaded successfully.


In [6]:
import gradio as gr
import torch
import numpy as np
import librosa
from transformers import ASTFeatureExtractor, ASTModel
from tensorflow.keras.models import load_model

# Load the AST feature extractor
feature_extractor = ASTFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

# Load the AST model
ast_model = ASTModel.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593").eval().to("cuda")


# Function to preprocess audio files using librosa and AST
def preprocess_audio(file):
    try:
        # Load audio with librosa and resample to 16kHz
        audio, sr = librosa.load(file, sr=16000)
        if sr != 16000:
            raise ValueError("Audio file is not resampled correctly to 16kHz.")

        # Use the AST feature extractor to process the audio
        inputs = feature_extractor(audio, sampling_rate=sr, return_tensors="pt", padding=True)
        inputs = inputs["input_values"].to("cuda")  # Move to GPU

        # Pass the preprocessed audio to the AST model to get embeddings
        with torch.no_grad():
            outputs = ast_model(inputs)  # Outputs from AST
            embeddings = outputs.last_hidden_state  # Shape: (batch_size, seq_length, feature_dim)

        # Perform temporal mean pooling to reduce the shape
        embeddings = torch.mean(embeddings, dim=1)  # Shape: (batch_size, 768)

        # Convert embeddings from PyTorch to NumPy
        embeddings = embeddings.cpu().detach().numpy()
        return embeddings
    except Exception as e:
        raise ValueError(f"Error during audio preprocessing: {str(e)}")

# Function to perform inference
def predict(file):
    try:
        # Preprocess the audio and run inference
        embeddings = preprocess_audio(file)
        prediction = model.predict(embeddings)  # TensorFlow model

        # Get the predicted class and probabilities
        predicted_class = np.argmax(prediction, axis=1)[0]
        predicted_category = metadata[metadata['target'] == predicted_class]['category'].values[0]

        # Get the top 5 classes and their probabilities
        top_5_indices = np.argsort(prediction[0])[::-1][:5]  # Get indices of top 5 probabilities
        top_5 = [
            {
                "category": metadata[metadata['target'] == i]['category'].values[0],
                "probability": prediction[0][i]
            }
            for i in top_5_indices
        ]

        # Format the result for display
        result = f"Predicted Category: {predicted_category}\n\nTop 5 Predictions:\n"
        for i, entry in enumerate(top_5):
            result += f"{i + 1}. {entry['category']}: {entry['probability']:.2f}\n"

        return result
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Set up the Gradio interface
demo = gr.Interface(
    fn=predict,
    inputs=gr.Audio(type="filepath"),  # Accept file path as input
    outputs="text",  # Output the predicted category and top 5 probabilities as text
    title="Audio Classification Demo - ESC-50",
    description="Upload an audio file and get the predicted category along with the top 5 probabilities."
)

# Launch the Gradio demo
demo.launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5e958f27fb03e00686.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
